<a href="https://colab.research.google.com/github/dh610/ai-intensive2/blob/main/lab8/token-classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/ai-intensive2
!git pull
%cd lab8

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ai-intensive2
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 25.86 KiB | 228.00 KiB/s, done.
From https://github.com/dh610/ai-intensive2
   f3819c4..c9516f9  main       -> origin/main
Updating f3819c4..c9516f9
Fast-forward
 lab8/token-classification.ipynb | 7067 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++-
 1 file changed, 7066 insertions(+), 1 deletion(-)
/content/drive/MyDrive/ai-intensive2/lab8


# BERT 토큰 분류 실습 - 개체명 인식 (Named Entity Recognition, NER)
![](https://raw.githubusercontent.com/nbroad1881/kaggle-images/main/feedback-prize-effectiveness/token-classification-diagram.png)
### 요구사항
1. token classification loss 구현하기
2. token classification metric 구현하기



In [4]:
!pip install -q transformers datasets

In [5]:
from pprint import pprint

## 데이터셋 살펴보기

In [6]:
from datasets import load_dataset

wnut = load_dataset("wnut_17")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

wnut_17.py:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

The repository for wnut_17 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wnut_17.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/3394 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1287 [00:00<?, ? examples/s]

In [7]:
# 문장이 word 단위로 쪼개져서 tokens에 list로 들어가있는 상태입니다.
# 이후 각 word마다 tag가 부여되어 있습니다.
# 하지만 우리는 tokenizer로 word를 subword로 분리해야 합니다 (예: @paulwalk -> @ ##paul ##walk로 분리될 수 있음)
# 그래서 전처리를 추가로 수행해 보겠습니다
pprint(wnut["train"][0])

{'id': '0',
 'ner_tags': [0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              7,
              8,
              8,
              0,
              7,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0],
 'tokens': ['@paulwalk',
            'It',
            "'s",
            'the',
            'view',
            'from',
            'where',
            'I',
            "'m",
            'living',
            'for',
            'two',
            'weeks',
            '.',
            'Empire',
            'State',
            'Building',
            '=',
            'ESB',
            '.',
            'Pretty',
            'bad',
            'storm',
            'here',
            'last',
            'evening',
         

In [9]:
# 각각의 정수 index가 어떤 class인지는 feature 필드에 저장되어 있습니다.
label_list = wnut["train"].features[f"ner_tags"].feature.names
label_list

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

ner tag는 각 word의 entity를 담고 있습니다. 하지만 B와 I라는 prefix가 있는데요.

- B: entity의 시작을 의미합니다.<br/>
- I: 이전 entity의 내부임을 의미합니다. 예를 들어, "Empire State Building" 이 띄어쓰기로 세 토큰으로 나뉜다고 했을 때, 이 세 토큰은 location에 해당합니다. 하지만 Empire는 B-location, State는 I-location이 됩니다. .<br/>
- 0: 어떤 entity에도 속하지 않는 것을 의미합니다. <br/>


전처리 과정 seudo code
```
tokens = ["Empire", "State", "Building"]
ner_tags = [7, 8, 8]

# 토크나이저가 아래처럼 subword로 쪼갰다고 가정해 봅시다.
tokenized = tokenizer(tokens) # ["Emp", "##ire", "State", "Build", "##ing"]

# 쪼개진 토큰이 원래 몇번째 word인지 알아낸 다음
words_ids = [0, 0, 1, 2, 2]

# ner tag를 쪼개진 토큰에 맞춰서 5개의 라벨로 늘려야 됩니다.

# 각 토큰을 13개의 ner tag 중 하나로 분류할 것이기 때문에, 라벨을 정수 인덱스로 바꿔주어야합니다.
labels = [7, 7, 8, 8, 8]

# 하지만 subword가 많아지면 특정 class가 너무 길게 이어질 수 있습니다. 그러면 학습 불균형과 오버피팅을 초래할 수 있습니다.
# 따라서 한 word에서 첫번째 subword를 제외한 나머지는 -100으로 처리해서 loss에서 제외시켜 불균형을 막아야 합니다.
labels = [7, -100, 8, 8, -100]
```

In [11]:
# bert-tiny가 작고 빨라요, 더 크고 뛰어난 모델을 쓰고 싶다면, 다른 사전학습 모델을 찾아 넣어봅시다.

# model_id = "distilbert-base-uncased"
model_id = "prajjwal1/bert-tiny"

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [13]:
example = wnut["train"][0]

# is_split_into_words=True 는 문자열이 아니라 이미 토큰으로 나눠진 list로 들어간다는 것을 의미합니다.
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)

# BERT model에 따라 token_type_ids가 없는 경우가 있습니다.
pprint(tokenized_input)

tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
pprint(tokens)

{'attention_mask': [1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1],
 'input_ids': [101,
               1030,
               2703,
               17122,
               2009,
               1005,
               1055,
               1996,
               3193,
               2013,
       

In [14]:
# word_ids는 각 토큰이 몇 번째 word인지를 의미합니다. CLS, SEP 같은 special token은 None으로 들어가 있습니다.
tokenized_input.word_ids(0)

[None,
 0,
 0,
 0,
 1,
 2,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 None]

쪼개진 토큰에 맞춰 라벨을 늘려봅시다. 또 special token은 label로 -100을 지정해서 학습에서 제외되도록 합시다.

In [34]:
# 빠른 처리를 위해 여러개를 한번에 하겠습니다(batch 처리)
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    print(tokenized_inputs[0])
    for i, label in enumerate(examples[f"ner_tags"]):
        # 원래 word id 가져오기
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_idx = 0

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # 구현해봅시다
            # 1. special tokens은 -100.
            # 2. subword로 분리될 경우 첫번째 토큰을 제외하고 -100
            ########
            ########
            if word_idx == None or word_idx == previous_word_idx:
                label_ids.append(-100)
            else:
                label_ids.append(label[label_idx])
                label_idx += 1

            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


test_batch = tokenize_and_align_labels(
    wnut["train"][0:4]
)
print(test_batch)


# 아래 값과 일치해야 정답
answer = {'input_ids': [[101, 1030, 2703, 17122, 2009, 1005, 1055, 1996, 3193, 2013, 2073, 1045, 1005, 1049, 2542, 2005, 2048, 3134, 1012, 3400, 2110, 2311, 1027, 9686, 2497, 1012, 3492, 2919, 4040, 2182, 2197, 3944, 1012, 102], [101, 2013, 2665, 2739, 7959, 2098, 1024, 6289, 7011, 8908, 15117, 2005, 10878, 2400, 2000, 13292, 1012, 1019, 8299, 1024, 1013, 1013, 4714, 3126, 2140, 1012, 4012, 1013, 2484, 8490, 3501, 22025, 102], [101, 1052, 20959, 2229, 2327, 2753, 5855, 5049, 4620, 1997, 2257, 2230, 1012, 1012, 1012, 8299, 1024, 1013, 1013, 2978, 1012, 1048, 2100, 1013, 1038, 18195, 2100, 2480, 2692, 1001, 5855, 102], [101, 2651, 2003, 2026, 2197, 2154, 2012, 1996, 2436, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, -100, -100, 0, 0, -100, 0, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, -100, 0, 0, 0, 0, 0, 0, 0, 0, -100], [-100, 0, 0, 0, -100, -100, 0, 5, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100], [-100, 1, -100, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, -100, 0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 0, -100, -100], [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]]}

def assert_list(a, b):
  assert len(a) == len(b)
  for aa, bb in zip(a, b):
    assert len(aa) == len(bb)
    assert all(x == y for x, y in zip(aa, bb))

for k, v in answer.items():
  assert_list(v, test_batch[k])

print("pass!")

Encoding(num_tokens=34, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
{'input_ids': [[101, 1030, 2703, 17122, 2009, 1005, 1055, 1996, 3193, 2013, 2073, 1045, 1005, 1049, 2542, 2005, 2048, 3134, 1012, 3400, 2110, 2311, 1027, 9686, 2497, 1012, 3492, 2919, 4040, 2182, 2197, 3944, 1012, 102], [101, 2013, 2665, 2739, 7959, 2098, 1024, 6289, 7011, 8908, 15117, 2005, 10878, 2400, 2000, 13292, 1012, 1019, 8299, 1024, 1013, 1013, 4714, 3126, 2140, 1012, 4012, 1013, 2484, 8490, 3501, 22025, 102], [101, 1052, 20959, 2229, 2327, 2753, 5855, 5049, 4620, 1997, 2257, 2230, 1012, 1012, 1012, 8299, 1024, 1013, 1013, 2978, 1012, 1048, 2100, 1013, 1038, 18195, 2100, 2480, 2692, 1001, 5855, 102], [101, 2651, 2003, 2026, 2197, 2154, 2012, 1996, 2436, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [35]:
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3394 [00:00<?, ? examples/s]

Encoding(num_tokens=34, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
Encoding(num_tokens=37, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
Encoding(num_tokens=56, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
Encoding(num_tokens=54, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

Encoding(num_tokens=14, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
Encoding(num_tokens=14, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

Encoding(num_tokens=31, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
Encoding(num_tokens=23, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [36]:
tokenized_wnut["train"][0]

{'id': '0',
 'tokens': ['@paulwalk',
  'It',
  "'s",
  'the',
  'view',
  'from',
  'where',
  'I',
  "'m",
  'living',
  'for',
  'two',
  'weeks',
  '.',
  'Empire',
  'State',
  'Building',
  '=',
  'ESB',
  '.',
  'Pretty',
  'bad',
  'storm',
  'here',
  'last',
  'evening',
  '.'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  8,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'input_ids': [101,
  1030,
  2703,
  17122,
  2009,
  1005,
  1055,
  1996,
  3193,
  2013,
  2073,
  1045,
  1005,
  1049,
  2542,
  2005,
  2048,
  3134,
  1012,
  3400,
  2110,
  2311,
  1027,
  9686,
  2497,
  1012,
  3492,
  2919,
  4040,
  2182,
  2197,
  3944,
  1012,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
 

Collator는 배치 크기를 통일시키고 padding을 추가해줍니다. label의 경우는 padding으로 -100이 추가됩니다.

In [37]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, padding=True, return_tensors="pt")

In [38]:
# "token_type_ids"는 BERT 모델에 따라 없는 경우도 있습니다.
# 필요한 BERT 모델의 경우에도 따로 추가해주지 않으면 huggingface model은 자체적으로 0으로 넣습니다.
# 이번 예제는 token_type_ids가 전부 0이므로 별도로 추가하지 않겠습니다.

allowed_keys = ["input_ids", "attention_mask", "labels"]
def filter_keys(d):
  return {k:v for k,v in d.items() if k in allowed_keys}

batch = [filter_keys(tokenized_wnut["train"][i]) for i in range(4)]
batch = data_collator(batch)
pprint(batch)

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'input_ids': tensor([[  101,  1030,  2703, 17122,  2009,  1005,  1055,  1996,  3193,  2013,
          2073,  1045,  1005,  1049,  2542,  2005,  2048,  3134,  1012,  3400,
          2110,  2311,  1027,  9686,  2497,  1012,  3492,  2919,  4040,  2182,
          2197,  3944,  1012,   102],
        [  101,  2013,  2665,  2739,  7959,  2098,  1024,  6289,  7011,  8908,
         15117,  2005, 10878,  2400,  2000, 13292,  1012,  1019,  8299,  1024,
          1013,  1013,  4714,  3126,  2140,  101

## 모델 설계

In [39]:
from transformers import AutoModel

# 사전학습된 모델 불러오기
model = AutoModel.from_pretrained(model_id)
model

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-1): 2 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=128, out_features=128, bias=True)
            (key): Linear(in_features=128, out_features=128, bias=True)
            (value): Linear(in_features=128, out_features=128, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=128, out_features=128, bias=True)
            (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)


In [40]:
# model의 출력값 확인 및 last_hidden_state 얻어오기
batch_no_labels = {k: v for k, v in batch.items() if k != "labels"}
model_output = model(**batch_no_labels)
pprint(model_output)
print(model_output.last_hidden_state.shape)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-1.8532, -0.1817, -4.4152,  ..., -0.8086, -1.0624,  0.5066],
         [-2.5370,  0.0360,  0.3745,  ..., -1.6066, -1.9431,  1.4169],
         [-1.0218, -0.2225, -0.6495,  ..., -1.3340, -0.2605,  0.3951],
         ...,
         [-1.7655, -0.5282,  0.2592,  ..., -2.0818, -0.6371,  0.1775],
         [-1.1009,  0.4529, -0.2139,  ..., -1.8122, -1.1135, -0.2835],
         [-2.8989,  0.6493,  0.0292,  ..., -1.5831, -0.5292,  0.6983]],

        [[-0.3251,  0.0589, -3.1188,  ..., -0.7906, -1.5180, -0.0530],
         [-0.4802,  0.7968,  0.0315,  ..., -1.4792, -1.0894,  0.8018],
         [-0.2377, -0.5652, -0.0903,  ..., -0.9559, -0.8842,  0.7802],
         ...,
         [-0.9495,  0.7532, -0.6763,  ..., -2.1269, -2.7322, -0.3240],
         [-1.0754, -0.0049,  0.6431,  ..., -1.8927, -2.5611,  0.9012],
         [-1.2750,  0.4552,  0.2146,  ..., -1.9369, -2.2242,  1.1257]],

        [[-0.2639,  1.2794, -3.0844,  ..., -1.7316, -

### Loss 계산
1. 각 토큰의 hidden state를 13개의 카테고리로 바꾼다 (nn.Linear)
2. cross-entropy를 계산한다
3. label이 -100인 값은 loss 계산에서 제외한다

In [41]:
id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
}
label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}
num_labels = 13

In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 모델의 hidden state 크기는 config에 있다.
model_hidden_size = model.config.hidden_size

X = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]).last_hidden_state
X = nn.Linear(model_hidden_size, num_labels, bias=False)(X)
print(X)
print(X.shape)

tensor([[[ 5.7060e-02,  1.8262e-02,  1.7774e+00,  ...,  1.6975e-01,
          -3.9049e-01,  4.4396e-02],
         [-1.6465e-01, -4.7464e-01,  5.4005e-01,  ...,  7.5820e-02,
          -6.5522e-01, -1.0361e+00],
         [ 4.1358e-01, -5.6450e-01,  1.0781e+00,  ..., -1.5645e-01,
          -4.2978e-02, -5.0235e-01],
         ...,
         [ 3.3375e-01, -2.5246e-04,  1.4617e+00,  ..., -1.1336e+00,
          -2.9243e-01, -8.7692e-02],
         [ 3.8274e-01, -1.6557e-01,  1.1592e+00,  ...,  1.4887e-01,
           8.1128e-01, -9.2949e-01],
         [-2.8057e-01, -2.2748e-01,  5.5581e-01,  ...,  5.2139e-01,
           2.3321e-01, -5.0759e-01]],

        [[ 1.3259e-01,  4.4820e-01,  9.1456e-01,  ..., -1.7691e-01,
          -4.4217e-02,  2.9849e-01],
         [ 3.3679e-01,  1.3263e-01,  1.4666e+00,  ..., -4.2686e-01,
           5.2148e-01, -2.0811e-01],
         [ 2.6970e-01, -3.2828e-01,  6.0032e-01,  ..., -6.0488e-01,
           7.8729e-01, -4.0694e-01],
         ...,
         [-1.8098e-01,  1

### loss 계산하는 함수를 직접 구현해보자

```
# 아래와 같은 logits, labels 있다고 했을 때
logits = [
  [ 0.1, 0.5, 0.4],
  [ 0.1, 0.4, 0.5],
  [ 0.5, 0.2, 0.3],
]
labels = [0, 2, -100]

# 최종 loss는 다음과 같아야 한다 (3번째는 label이 -100이라서 무시)
loss = sum(log(0.1) + log(0.5) + 0) / 2
```

계산 방법
1. `torch.gather`를 이용해서, 정답 index의 값을 빼온다
1. log를 취해서 loglikelihood를 계산한다
1. labels가 -100인 값들을 제외하고 평균을 취한다: masked_fill을 이용해서, -100이면 0, 아니면 1인 mask 텐서를 만든다. 이후 loglikelihood에 곱해서 0으로 바꾸면 된다.




In [43]:
# torch.gather & masked_fill 예시,

logits = torch.tensor([
  [ 0.1, 0.5, 0.4],
  [ 0.1, 0.4, 0.5],
  [ 0.5, 0.2, 0.3],
])
labels = torch.tensor([0, 2, -100])

# -100 인 label은 gather할 경우 에러가 난다. 따라서 0으로 바꿔준다 - 아무 값으로 바꿔도 상관없다. 나중에 loss 합칠 때 제외하면 된다
labels_nonzero = labels.masked_fill(labels < 0, 0)

print("labels_nonzero", labels_nonzero)
gathered_logits = torch.gather(logits, -1, labels_nonzero.unsqueeze(-1))
print("gathered_logits", gathered_logits)

labels_mask = (labels >= 0).float()
print("labels_mask", labels_mask)
print("-100인 label은 log loss에서 0으로 만들기", gathered_logits.log().squeeze(-1) * labels_mask)

labels_nonzero tensor([0, 2, 0])
gathered_logits tensor([[0.1000],
        [0.5000],
        [0.5000]])
labels_mask tensor([1., 1., 0.])
-100인 label은 log loss에서 0으로 만들기 tensor([-2.3026, -0.6931, -0.0000])


In [92]:
# cross-entropy 계산
def token_classification_loss(logits, labels):
  # logits: shape[batch, seq, labels]
  # labels: shape[batch, seq]

  logits = logits.softmax(-1)

  ######## 구현해봅시당~~~~
  mask = labels != -100
  labels_nonzero = labels.masked_fill(labels < 0, 0)
  gathered_logits = torch.gather(logits, -1, labels_nonzero.unsqueeze(-1))
  masked_logits = gathered_logits[mask]
  loss = -torch.log(masked_logits).mean()

  return loss.item()

# 2.6628 정답
token_classification_loss(X, batch["labels"])

2.5222692489624023

model과 classification head를 합쳐서 하나의 모듈로 만들어봅시다

In [93]:
from typing import Optional

class TokenClassificationModule(nn.Module):
  def __init__(self, pretrained_model_id: str, num_classes: int):
    super().__init__()

    self.model = AutoModel.from_pretrained(pretrained_model_id)
    self.class_head = nn.Linear(self.model.config.hidden_size, num_classes, bias=False)

  def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor):
    X = self.model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
    X = self.class_head(X)
    return X

model = TokenClassificationModule(model_id, num_labels)
model(**batch_no_labels)

tensor([[[-0.3666, -0.2998, -0.0102,  ..., -0.7180,  0.6375, -1.2840],
         [ 0.3803, -0.6741, -0.5301,  ..., -1.1508,  0.5506, -0.7948],
         [ 0.2015,  0.2597, -0.0429,  ..., -0.4818,  0.8655, -0.6914],
         ...,
         [ 0.0517, -0.8689, -0.1971,  ..., -0.2113,  0.1422, -0.2437],
         [ 0.3717, -0.2562, -0.0544,  ..., -0.2430,  0.9031,  0.1309],
         [ 0.9160, -0.8784, -0.6432,  ..., -0.4716,  0.9476,  0.0140]],

        [[-0.4351, -0.3488, -0.4876,  ..., -0.6430,  1.4038, -0.2603],
         [ 0.4263, -0.6036, -0.9726,  ...,  0.0453,  0.8403, -0.4157],
         [ 0.0249, -0.1634, -0.4894,  ...,  0.1409,  0.9202, -0.1425],
         ...,
         [ 0.8738,  0.0426, -0.3880,  ..., -0.6083,  1.0462, -0.1399],
         [ 0.7877, -0.6383, -0.5861,  ..., -0.3782,  1.0653,  0.2288],
         [ 0.8189, -0.4915, -1.0570,  ..., -0.1115,  1.4717,  0.2745]],

        [[-0.4550,  0.1097, -0.0830,  ..., -0.1114,  1.3216, -0.4646],
         [ 0.0501,  0.2056, -0.6044,  ..., -0

### 평가 metric 계산

In [94]:
def token_accuracy(logits, labels):
  # logits: [batch, seq_len, num_labels]
  # labels; [batch, seq_len]

  # 구현해봅시다!!!!
  # logits에서 가장 높은 확률을 가진 index가 label과 일치하는지 확인해야합니다.
  # 일치하는 값의 비율이 accuracy가 됩니다.
  # 단, labels가 -100인 경우를 제외해야 합니다.
  preds = logits.argmax(-1)
  mask = labels != -100
  correct = (preds == labels) & mask
  acc_sum = correct.sum().item()
  acc_count = mask.sum().item()

  return acc_sum / acc_count

assert token_accuracy(
    torch.tensor ([
        [0.1, 0.8, 0.1],
        [0.1, 0.8, 0.1],
        [0.2, 0.4, 0.4]
    ]),
    torch.tensor ([
        [
            0,
            1,
            -100
        ]
    ])
) == 0.5 # 정답 0.5

In [128]:

def token_macro_f1(logits, labels):
  num_labels = logits.shape[-1]
  preds = logits.argmax(-1)

  micro_f1 = []
  total_labels = (labels != -100).sum()
  mask = labels != -100

  for i in range(num_labels):
    ### 구현해봅시다!!!! (-100 제외하고 구하기!)
    valid_class_mask = (labels == i) & mask
    if labels[0][i] == -100:
      continue
    tp = ((preds == i) & valid_class_mask).sum().item()
    fp = ((preds == i) & (~valid_class_mask) & mask).sum().item()
    fn = ((preds != i) & valid_class_mask).sum().item()

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    micro_f1.append(f1)

  return torch.tensor(micro_f1).mean().item()


assert token_macro_f1(
    torch.tensor ([
        [0.1, 0.8, 0.1],
        [0.1, 0.8, 0.1],
        [0.2, 0.4, 0.4]
    ]),
    torch.tensor ([
        [
            0,
            1,
            -100
        ]
    ])
) == 0.5 # 정답 0.5

0.3333333432674408


AssertionError: 

# 학습해보기

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

batch_size = 8

filtered_data = tokenized_wnut.map(filter_keys, remove_columns=["id", "tokens", "ner_tags"])
training_loader = torch.utils.data.DataLoader(filtered_data["train"], batch_size=batch_size, shuffle=True, collate_fn=data_collator)
validation_loader = torch.utils.data.DataLoader(filtered_data["validation"], batch_size=batch_size, shuffle=False, collate_fn=data_collator)

In [ ]:
# DataLoader 의 값 살펴보기
next(iter(training_loader))

{'input_ids': tensor([[  101,  2651,  1005,  1055, 15138,  2253,  2013,  9643,  2000, 12476,
          2013,  1996,  2617,  1045,  3651,  1045,  2106,  2025,  2031,  2465,
          2651,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  2431, 17776,  1010,  2279,  2129,  2152,  1010, 13055,  3337,
          1010, 23893, 23893,  3413,  1010,  1998,  1018,  2847,  1997,  5958,
          1012,  1004,  8318,  1025,  2757,  2665,  7530,  2015,  1004, 14181,
          1025,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101, 19387,  1030,  6788,  2015, 20554,  1024,  2893,  3201,  2005,
          2256,  2345,  2047,  2265,  1997,  2297,  1012,  6069,  2022,  1037,
          2204,  2028,  1011,  1011,

In [ ]:
from tqdm import tqdm

def change_dict_device(x):
  return {k: v.to(device) for k, v in x.items()}

def train_epoch(epoch_index):
    running_loss, running_acc, running_macro_f1 = 0.0, 0.0, 0.0
    last_loss = 0.
    log_freq = 50

    for i, batch in enumerate(tqdm(training_loader, desc=f"Epoch {epoch_index}")):
        optimizer.zero_grad()

        change_dict_device(batch)

        input_ids, attention_mask, labels = batch["input_ids"], batch["attention_mask"], batch["labels"]

        logits = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        loss = token_classification_loss(logits, labels)
        loss.backward()

        optimizer.step()

        running_loss += loss.item()

        if (i + 1) % log_freq == 0:
            last_loss = running_loss / log_freq
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            running_loss = 0.

    return last_loss

@torch.no_grad()
def validation_epoch(epoch_index):
    running_loss, running_acc, running_macro_f1 = 0.0, 0.0, 0.0
    total_logits, total_labels = [], []
    for i, batch in enumerate(tqdm(validation_loader, desc=f"Evaluating...", leave=False)):
        change_dict_device(batch)

        input_ids, attention_mask, labels = batch["input_ids"], batch["attention_mask"], batch["labels"]

        logits = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        loss = token_classification_loss(logits, labels)

        running_loss += loss.item()

        total_logits.extend(logits.view(-1, 13).tolist()) # 1차원으로 바꿔서 추가
        total_labels.extend(labels.view(-1).tolist()) # 1차원으로 바꿔서 추가

    running_loss = running_loss / i
    total_logits, total_labels = torch.tensor(total_logits), torch.tensor(total_labels)

    running_acc = token_accuracy(total_logits, total_labels)
    running_macro_f1 = token_macro_f1(total_logits, total_labels)

    print(f"evaluation #{epoch_index}: loss={running_loss}, accuracy={running_acc}, macro-f1={running_macro_f1}")

In [ ]:
EPOCHS = 1

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch))

    model.train()
    avg_loss = train_epoch(epoch)
    print(f"LOSS train {avg_loss}")

    model.eval()
    validation_epoch(epoch)

EPOCH 0:


Epoch 0:  12%|█▏        | 50/425 [02:19<15:59,  2.56s/it]

  batch 50 loss: 0.46735170535743237


Epoch 0:  24%|██▎       | 100/425 [04:34<14:09,  2.61s/it]

  batch 100 loss: 0.27528622748330234


...

Epoch 0:  94%|█████████▍| 400/425 [18:09<01:11,  2.86s/it]

  batch 400 loss: 0.13688919843174518


Epoch 0: 100%|██████████| 425/425 [19:16<00:00,  2.72s/it]


LOSS train 0.13688919843174518


evaluation #0: loss=0.268198549976602, accuracy=0.9467861652374268, macro-f1=0.2777317762374878


# 실제로 사용해보기


In [ ]:

def predict(text):
  inputs = tokenizer(text, return_tensors="pt", is_split_into_words=True).to(device)
  tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

  logits = model(**inputs)

  preds = logits.argmax(-1).tolist()
  preds = [id2label[i] for i in preds[0]]

  for token, pred in zip(tokens, preds):
    print("prediction: ", token, pred)

predict(tokenized_wnut["validation"][1]["tokens"])

prediction:  [CLS] B-person
prediction:  you O
prediction:  should O
prediction:  ' O
prediction:  ve O
prediction:  stayed O
prediction:  on O
prediction:  red B-location
prediction:  ##ond I-location
prediction:  ##o I-location
prediction:  beach I-location
prediction:  b O
prediction:  ##l O
prediction:  ##vd O
prediction:  . O
prediction:  you O
prediction:  were O
prediction:  in O
prediction:  the O
prediction:  border O
prediction:  ##lines O
prediction:  of O
prediction:  garden B-location
prediction:  ##a I-location
prediction:  / O
prediction:  compton B-location
prediction:  [SEP] B-person
